Important steps to do before running the code blocks below!! 🚀🚀
1. Create a data folder and upload your csv data into.
2. Create a pieline folder to store metadata from the various components

In [1]:
!mkdir ./data
!mkdir ./pipeline

In [2]:
!pip install tfx

     |████████████████████████████████| 2.4 MB 5.3 MB/s 
     |████████████████████████████████| 1.7 MB 49.2 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
     |████████████████████████████████| 205 kB 74.2 MB/s 
     |████████████████████████████████| 98 kB 5.9 MB/s 
     |████████████████████████████████| 40 kB 4.5 MB/s 
     |████████████████████████████████| 1.2 MB 39.6 MB/s 
     |████████████████████████████████| 422 kB 51.9 MB/s 
     |████████████████████████████████| 147 kB 45.4 MB/s 
     |████████████████████████████████| 6.6 MB 40.4 MB/s 
     |████████████████████████████████| 19.1 MB 1.2 MB/s 
     |████████████████████████████████| 1.8 MB 51.6 MB/s 
     |████████████████████████████████| 1.7 MB 45.9 MB/s 
     |████████████████████████████████| 1.4 MB 49.3 MB/s 
     |████████████████████████████████| 135 kB 61.2 MB/s 
     |████████████████████████████████| 9.9 MB 49.0 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |█████████████████

In [3]:
!pip install dask[complete]

     |████████████████████████████████| 132 kB 5.3 MB/s 
     |████████████████████████████████| 802 kB 49.8 MB/s 
     |████████████████████████████████| 802 kB 49.1 MB/s 
     |████████████████████████████████| 793 kB 54.1 MB/s 
     |████████████████████████████████| 793 kB 45.6 MB/s 
     |████████████████████████████████| 791 kB 49.4 MB/s 
     |████████████████████████████████| 786 kB 50.9 MB/s 
     |████████████████████████████████| 779 kB 54.4 MB/s 
     |████████████████████████████████| 778 kB 52.2 MB/s 
     |████████████████████████████████| 776 kB 70.0 MB/s 
     |████████████████████████████████| 769 kB 59.0 MB/s 
     |████████████████████████████████| 766 kB 54.8 MB/s 
     |████████████████████████████████| 1.0 MB 52.9 MB/s 
     |████████████████████████████████| 722 kB 65.5 MB/s 
     |████████████████████████████████| 722 kB 63.1 MB/s 
     |████████████████████████████████| 715 kB 59.7 MB/s 
     |████████████████████████████████| 705 kB 61.1 MB/s 
     |█████████

In [1]:
from tfx.components import CsvExampleGen
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

import tensorflow as tf
import os
import pprint
import dask.dataframe as dd

pp = pprint.PrettyPrinter()


In [2]:
context = InteractiveContext(pipeline_root='pipeline')

# 1. Data Cleaning with Dask 

In [3]:
df1 = dd.read_csv('data/*.csv')
df1.head()

,username,rating,helpful,total,date,title,review
0,Imme-van-Gorp,7,102,123,30 January 2019,Unfortunately the ending ruined an otherwise ...,This movie is full of suspense. It makes you g...
1,sonofocelot-1,5,385,500,10 May 2016,...oh dear Abrams. Again.\n,I'll leave this review fairly concise. <br/><b...
2,mhodaee,5,110,143,4 August 2017,"Fantastic, gripping, thoroughly enjoyable, un...",I give the 5/10 out of the credit I owe to the...
3,fil-nik09,5,73,100,5 October 2016,Hmmm...\n,"First of all, I must say that I was expecting ..."
4,DVR_Brale,7,42,56,27 July 2016,Slow building & plot alternating claustrophob...,I've always loved movies with strong atmospher...


In [4]:
#dropping unnecessary columns from the dataset
df2 = df1[['review', 'rating']]
df2.head()

,review,rating
0,This movie is full of suspense. It makes you g...,7
1,I'll leave this review fairly concise. <br/><b...,5
2,I give the 5/10 out of the credit I owe to the...,5
3,"First of all, I must say that I was expecting ...",5
4,I've always loved movies with strong atmospher...,7


In [5]:
#drop all rows with the null rating
df3 = df2[df2.rating != 'Null']
df3.head()

,review,rating
0,This movie is full of suspense. It makes you g...,7
1,I'll leave this review fairly concise. <br/><b...,5
2,I give the 5/10 out of the credit I owe to the...,5
3,"First of all, I must say that I was expecting ...",5
4,I've always loved movies with strong atmospher...,7


In [6]:
#Change all ratings to sentiment types
#ratings are initially strings so we need to change them to integers

df3.rating = df3.rating.astype('int')
df3.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 2 entries, review to rating
dtypes: object(1), int64(1)

In [7]:
df3['sentiment'] = (df3.rating > 5).astype('int')
df3.head()

,review,rating,sentiment
0,This movie is full of suspense. It makes you g...,7,1
1,I'll leave this review fairly concise. <br/><b...,5,0
2,I give the 5/10 out of the credit I owe to the...,5,0
3,"First of all, I must say that I was expecting ...",5,0
4,I've always loved movies with strong atmospher...,7,1


In [35]:
#convert dataframe to csv
df4 = df3[['review', 'sentiment']]
df4.to_csv('./data_in.csv', index=False)

['/content/./data_in.csv/00.part',
 '/content/./data_in.csv/01.part',
 '/content/./data_in.csv/02.part',
 '/content/./data_in.csv/03.part',
 '/content/./data_in.csv/04.part',
 '/content/./data_in.csv/05.part',
 '/content/./data_in.csv/06.part',
 '/content/./data_in.csv/07.part',
 '/content/./data_in.csv/08.part',
 '/content/./data_in.csv/09.part',
 '/content/./data_in.csv/10.part',
 '/content/./data_in.csv/11.part',
 '/content/./data_in.csv/12.part',
 '/content/./data_in.csv/13.part',
 '/content/./data_in.csv/14.part',
 '/content/./data_in.csv/15.part',
 '/content/./data_in.csv/16.part',
 '/content/./data_in.csv/17.part',
 '/content/./data_in.csv/18.part',
 '/content/./data_in.csv/19.part',
 '/content/./data_in.csv/20.part',
 '/content/./data_in.csv/21.part',
 '/content/./data_in.csv/22.part',
 '/content/./data_in.csv/23.part',
 '/content/./data_in.csv/24.part',
 '/content/./data_in.csv/25.part',
 '/content/./data_in.csv/26.part',
 '/content/./data_in.csv/27.part',
 '/content/./data_in

In [36]:
df4.head()

review  sentiment
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        This movie is full of suspense. It makes you guess about what is real and what is not. It happens more than once that you have to wonder about what is the truth and who is lying.<br/><br/>Because you are just as clueless as the main character, Michelle, you really get to experience the same type of emotions and confusion as she is. This not only makes you feel closer to Michelle as a character, but also to the overall story.<br/><br/>They did, however, ruin this connection that they build over the course of the story by making 

# 2. Pipeline

## 2.1 Data Ingestion

## 2.1.1 ExampleGen

In [9]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2)
        ])
)

In [ ]:
#!mv data/sample_data/ ./

In [10]:
example_gen = CsvExampleGen(input_base='data_in.csv', output_config=output)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "pipeline/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:96,total_bytes:105470260,xor_checksum:13,sum_checksum:157578131821"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        base_type: DATASET
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [11]:
train_url = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')
#eval_url = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-eval')

In [12]:
tfrecord_filenames = [os.path.join(train_url, name) for name in os.listdir(train_url)]
#ev_tfrecord_filenames = [os.path.join(eval_url, name) for name in os.listdir(eval_url)]

In [13]:
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type='GZIP')
#ds_eval = tf.data.TFRecordDataset(ev_tfrecord_filenames, compression_type='GZIP')

In [ ]:
for tfrecord in dataset.take(2):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)


In [ ]:
#for tfrecord in ds_eval.take(2):
#  ev_serialized_example = tfrecord.numpy()
#  ev_example = tf.train.Example()
#  ev_example.ParseFromString(ev_serialized_example)
#  pp.pprint(ev_example)

# 2.2 Data Validation

## 2.2.1 StatisticsGen

In [15]:
from tfx.components import StatisticsGen


In [16]:
statistics_gen = StatisticsGen(
    examples= example_gen.outputs['examples']
)
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 16
        uri: "pipeline/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        base_type: STATISTICS
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [17]:
context.show(statistics_gen.outputs['statistics'])

## 2.2.2 SchemaGen

In [18]:
from tfx.components import SchemaGen

In [19]:
schema_gen = SchemaGen(
    statistics = statistics_gen.outputs['statistics']
)

In [20]:
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "pipeline/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [21]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'review',BYTES,required,,-
'sentiment',INT,required,,-


## 2.2.3 ExampleValidator

In [22]:
from tfx.components import ExampleValidator

In [23]:
example_validator = ExampleValidator(
    statistics = statistics_gen.outputs['statistics'],
    schema = schema_gen.outputs['schema']
)

In [24]:
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 20
        uri: "pipeline/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [25]:
context.show(example_validator.outputs['anomalies'])

# 2.3 Data Preprocessing

## 2.3.1 Transform

NLP tasks need the following transforms
- change all the text to lowercase
- remove html tags
- stop words
- remove punctuations

In [26]:
_transform_module_file = '_transform.py'

In [44]:
%%writefile {_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft


stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", 
             "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", 
             "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", 
             "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", 
             "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", 
             "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", 
             "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", 
             "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", 
             "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", 
             "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", 
             "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

_LABEL_KEY = 'sentiment'

#Renaming all transform features
def _transformed_name(key):
  return key + '_xf'

#Define the transformations. The name is fixed since that is what is expected
def preprocessing_fn(inputs):
  outputs = {}
  #Change all the texts to lowercase
  outputs[_transformed_name('review')] = tf.strings.lower(inputs['review'])

  return outputs


Overwriting _transform.py


In [45]:
#run the transform component
from tfx.components import Transform

transform = Transform(
    examples= example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=_transform_module_file
)


In [46]:
context.run(transform)

INFO:tensorflow:Assets written to: pipeline/Transform/transform_graph/8/.temp_path/tftransform_tmp/e71a27b45a784d5484f7420e03fe8d0d/assets


INFO:tensorflow:Assets written to: pipeline/Transform/transform_graph/8/.temp_path/tftransform_tmp/e71a27b45a784d5484f7420e03fe8d0d/assets


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 8
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 29
        type_id: 22
        uri: "pipeline/Transform/transform_graph/8"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 30
        type_id: 14
        uri: "pipeline/Transform/transformed_examples/8"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        base_type: DATASET
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 31
        type_id: 23
        uri: "pipeline/Transform/updated_analyzer_cache/8"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 32
        type_id: 18
        uri: "pipeline/Transform/pre_transform_schema/8"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        

Showing the transformed outputs

In [47]:
train_url = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')
tfrecord_filenames = [os.path.join(train_url, name) for name in os.listdir(train_url)]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type='GZIP')
for tfrecord in dataset.take(2):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "review_xf"
    value {
      bytes_list {
        value: "i\'m a 17 year old male teenager who happened to stumble upon this movie two years ago at school during a free day in math class (when i was 15). i liked it then, but the math class ended before the movie could be finished. finally, on the wb a few days ago, the movie aired and i saw the conclusion. awtr contains a heartfelt conclusion to a story where you get to care about landon and jamie, two polar opposite teenagers who face peer struggles and the perils of love and death. the situations that occur are very real, and very sad. once the movie finishes, you\'ll be left wondering what happens to everyone (heck, even though it doesn\'t make sense, i want a sequel!).<br/><br/>as a maturing male, i couldn\'t help but cry inside, and i\'m someone who felt indifferent toward movies like titanic. i found myself wishing i were either landon or jamie, because they represent the real virtues of life and 